# Execution Engine Demo: Computing Mechanical Parameters via All Possible Pathways

This notebook demonstrates the new **Execution Engine** feature that automatically computes mechanical parameters using all possible calculation pathways defined in the parameterization graph.

## Key Concepts

- **Parameterization Graph**: A directed acyclic graph (DAG) that defines all possible ways to calculate mechanical parameters from measured inputs
- **Pathway**: A specific sequence of methods from measured inputs to a target parameter
- **ExecutionEngine**: Executes all pathways and returns results for each

## What This Notebook Covers

1. Creating Layer and Slab objects from measured data
2. Listing all available calculation pathways
3. Executing all pathways for a target parameter
4. Comparing results across different pathways
5. Using real SnowPilot data with the execution engine

In [2]:
# Import standard libraries
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import from uncertainties for creating measured values with uncertainty
from uncertainties import ufloat

# Import snowpyt_mechparams components (updated paths)
from snowpyt_mechparams.data_structures import Layer, Slab, Pit
from snowpyt_mechparams.execution import ExecutionEngine, ExecutionResults, PathwayResult
from snowpyt_mechparams.graph import graph

# Import SnowPilot utilities (now part of snowpyt_mechparams package)
from snowpyt_mechparams.snowpilot_utils import parse_caaml_file

print("Imports successful!")

Imports successful!


## 1. Creating Test Data

First, let's create a simple slab with measured data. We'll include:
- Hand hardness (for density estimation)
- Grain form (used by multiple methods)
- Optional: Direct density measurements

In [3]:
# Create a 3-layer slab with mixed measurement availability
layers = [
    # Layer 0: Surface layer with direct density measurement
    Layer(
        depth_top=0,
        thickness=15,
        hand_hardness='1F',
        grain_form='RG',  # Rounded grains
        grain_size_avg=ufloat(1.0, 0.1),
        density_measured=ufloat(280, 25)  # kg/m³
    ),
    # Layer 1: Middle layer - no direct density, will be estimated
    Layer(
        depth_top=15,
        thickness=20,
        hand_hardness='P',
        grain_form='FC',  # Faceted crystals
        grain_size_avg=ufloat(2.0, 0.2)
    ),
    # Layer 2: Bottom layer with direct density
    Layer(
        depth_top=35,
        thickness=25,
        hand_hardness='4F',
        grain_form='DF',  # Decomposing/fragmented
        grain_size_avg=ufloat(0.5, 0.05),
        density_measured=ufloat(200, 18)
    )
]

# Create the slab with a 35° slope angle
test_slab = Slab(layers=layers, angle=35.0)

print(f"Created slab with {len(test_slab.layers)} layers")
print(f"Total thickness: {test_slab.total_thickness} cm")
print(f"Slope angle: {test_slab.angle}°")
print()

# Display layer details
for i, layer in enumerate(test_slab.layers):
    print(f"Layer {i}:")
    print(f"  Depth: {layer.depth_top} - {layer.depth_bottom} cm")
    print(f"  Hand hardness: {layer.hand_hardness}")
    print(f"  Grain form: {layer.grain_form}")
    print(f"  Density measured: {layer.density_measured}")
    print()

Created slab with 3 layers
Total thickness: 60 cm
Slope angle: 35.0°

Layer 0:
  Depth: 0 - 15 cm
  Hand hardness: 1F
  Grain form: RG
  Density measured: 280+/-25

Layer 1:
  Depth: 15 - 35 cm
  Hand hardness: P
  Grain form: FC
  Density measured: None

Layer 2:
  Depth: 35 - 60 cm
  Hand hardness: 4F
  Grain form: DF
  Density measured: 200+/-18



## 2. Initialize the Execution Engine

The `ExecutionEngine` takes the parameterization graph and provides methods to:
- List all available pathways for a target parameter
- Execute all pathways on a slab
- Execute a specific pathway by method names

In [4]:
# Initialize the execution engine with the graph
engine = ExecutionEngine(graph)

print("Execution engine initialized.")
print("\nAvailable target parameters:")
print("  - density")
print("  - elastic_modulus")
print("  - poissons_ratio")
print("  - shear_modulus")

Execution engine initialized.

Available target parameters:
  - density
  - elastic_modulus
  - poissons_ratio
  - shear_modulus


## 3. List Available Pathways

Before executing, let's see all the possible pathways for computing elastic modulus.

In [5]:
# List all available pathways for elastic_modulus
pathways = engine.list_available_pathways('elastic_modulus')

print(f"Found {len(pathways)} possible pathways for computing elastic_modulus:\n")

for i, pathway in enumerate(pathways, 1):
    print(f"{i:2d}. {pathway['description']}")
    print(f"    Methods: {pathway['methods']}")
    print()

Found 16 possible pathways for computing elastic_modulus:

 1. density=data_flow | elastic_modulus=bergfeld
    Methods: {'density': 'data_flow', 'elastic_modulus': 'bergfeld'}

 2. density=geldsetzer | elastic_modulus=bergfeld
    Methods: {'density': 'geldsetzer', 'elastic_modulus': 'bergfeld'}

 3. density=kim_jamieson_table2 | elastic_modulus=bergfeld
    Methods: {'density': 'kim_jamieson_table2', 'elastic_modulus': 'bergfeld'}

 4. density=kim_jamieson_table5 | elastic_modulus=bergfeld
    Methods: {'density': 'kim_jamieson_table5', 'elastic_modulus': 'bergfeld'}

 5. density=data_flow | elastic_modulus=kochle
    Methods: {'density': 'data_flow', 'elastic_modulus': 'kochle'}

 6. density=geldsetzer | elastic_modulus=kochle
    Methods: {'density': 'geldsetzer', 'elastic_modulus': 'kochle'}

 7. density=kim_jamieson_table2 | elastic_modulus=kochle
    Methods: {'density': 'kim_jamieson_table2', 'elastic_modulus': 'kochle'}

 8. density=kim_jamieson_table5 | elastic_modulus=kochle

## 4. Execute All Pathways

Now let's execute all pathways and see the results. The engine will:
1. Find all parameterizations for the target parameter
2. Execute each one on a deep copy of the slab
3. Return results keyed by pathway description

In [6]:
# Execute all pathways for elastic_modulus
results = engine.execute_all(
    slab=test_slab,
    target_parameter='elastic_modulus'
)

print("=== Execution Summary ===")
print(f"Target parameter: {results.target_parameter}")
print(f"Total pathways attempted: {results.total_pathways}")
print(f"Successful pathways: {results.successful_pathways}")
print(f"Cache hit rate: {results.cache_stats['hit_rate']:.1%}")

TypeError: ExecutionEngine.execute_all() got an unexpected keyword argument 'include_plate_theory'

## 5. Explore Results by Pathway

Each pathway result contains:
- `methods_used`: Dictionary of parameter → method name
- `layer_results`: List of LayerResult objects with computed values
- `slab_result`: SlabResult with plate theory parameters (A11, B11, D11, A55)
- `success`: Whether the pathway produced results

In [ ]:
# Explore the first 3 successful pathways in detail
count = 0
for pathway_desc, pathway_result in results.pathways.items():
    if pathway_result.success and count < 3:
        print(f"\n{'='*60}")
        print(f"Pathway: {pathway_desc}")
        print(f"{'='*60}")
        print(f"\nMethods used: {pathway_result.methods_used}")
        print(f"\nLayer Results:")
        
        slab = pathway_result.slab
        for i, layer in enumerate(slab.layers):
            print(f"\n  Layer {i} (depth {layer.depth_top}-{layer.depth_bottom} cm):")
            print(f"    Grain form: {layer.grain_form}")
            
            if layer.density_calculated is not None:
                print(f"    Density (calculated): {layer.density_calculated:.1f} kg/m³")
            elif layer.density_measured is not None:
                print(f"    Density (measured): {layer.density_measured:.1f} kg/m³")
            
            if layer.elastic_modulus is not None:
                print(f"    Elastic modulus: {layer.elastic_modulus:.2f} MPa")
            else:
                print(f"    Elastic modulus: Not computed")
        
        # Show computation traces
        print(f"\n  Computation traces (first 5):")
        layer_traces = pathway_result.get_layer_traces()[:5]
        for trace in layer_traces:
            status = "✓" if trace.success else "✗"
            cached_marker = "[CACHED]" if trace.cached else ""
            print(f"    {status} Layer {trace.layer_index}: {trace.parameter}.{trace.method_name} {cached_marker}")
            if not trace.success and trace.error:
                print(f"      Error: {trace.error}")
        
        count += 1

## 6. Compare Results Across Pathways

Let's create a comparison table showing how elastic modulus values vary by pathway and layer.

In [ ]:
# Build comparison data
comparison_data = []

for pathway_desc, pathway_result in results.pathways.items():
    if pathway_result.success:
        slab = pathway_result.slab
        
        # Get method names
        density_method = pathway_result.methods_used.get('density', 'N/A')
        emod_method = pathway_result.methods_used.get('elastic_modulus', 'N/A')
        
        for i, layer in enumerate(slab.layers):
            if layer.elastic_modulus is not None:
                comparison_data.append({
                    'Layer': i,
                    'Grain Form': layer.grain_form,
                    'Density Method': density_method,
                    'E Method': emod_method,
                    'E (MPa)': layer.elastic_modulus.nominal_value,
                    'E Uncertainty (MPa)': layer.elastic_modulus.std_dev
                })

# Create DataFrame
if comparison_data:
    df = pd.DataFrame(comparison_data)
    
    # Display pivot table grouped by layer
    print("Elastic Modulus Comparison by Layer and Method Combination:\n")
    pivot = df.pivot_table(
        values='E (MPa)',
        index=['Layer', 'Grain Form'],
        columns=['Density Method', 'E Method'],
        aggfunc='mean'
    )
    print(pivot.round(2).to_string())
else:
    print("No successful elastic modulus calculations to compare.")

## 7. Execute a Specific Pathway

If you want to run only a specific combination of methods, use `execute_single()`.

In [ ]:
# To execute a specific pathway, filter results for the desired method combination
specific_results = engine.execute_all(
    slab=test_slab,
    target_parameter='elastic_modulus'
)

# Find the pathway that uses data_flow for density and wautier for elastic modulus
found = False
for pathway_desc, pathway_result in specific_results.pathways.items():
    methods = pathway_result.methods_used
    if methods.get('density') == 'data_flow' and methods.get('elastic_modulus') == 'wautier':
        print(f"Executed pathway: {pathway_desc}")
        print(f"Success: {pathway_result.success}")
        print(f"\nMethods used: {methods}")
        print("\nLayer results:")
        
        slab = pathway_result.slab
        for i, layer in enumerate(slab.layers):
            print(f"\n  Layer {i}:")
            density = layer.density_calculated if layer.density_calculated else layer.density_measured
            print(f"    Density: {density}")
            print(f"    Elastic modulus: {layer.elastic_modulus}")
        
        # Check for slab-level parameters
        if slab.D11:
            print(f"\nSlab-level parameters (if computed):")
            print(f"  D11 (bending stiffness): {slab.D11:.1f} N·mm")
        
        found = True
        break

if not found:
    print("Specific pathway not found or failed.")

## 8. Computing Shear Modulus with Plate Theory

Let's compute shear modulus which includes the A55 plate theory parameter.

In [ ]:
# Execute for shear modulus
shear_results = engine.execute_all(
    slab=test_slab,
    target_parameter='shear_modulus'
)

print(f"Shear modulus pathways: {shear_results.total_pathways}")
print(f"Successful: {shear_results.successful_pathways}")
print(f"Cache hit rate: {shear_results.cache_stats['hit_rate']:.1%}")

# Show first successful result
for pathway_desc, pathway_result in shear_results.pathways.items():
    if pathway_result.success:
        print(f"\n=== {pathway_desc} ===")
        print(f"\nLayer shear modulus values:")
        
        slab = pathway_result.slab
        for i, layer in enumerate(slab.layers):
            if layer.shear_modulus is not None:
                print(f"  Layer {i}: G = {layer.shear_modulus:.2f} MPa")
        
        # Check for A55 slab parameter
        if slab.A55 is not None:
            print(f"\nSlab shear stiffness (κ*A55): {slab.A55:.2f} N/mm")
        
        break  # Just show first

## 9. Working with Real SnowPilot Data

Now let's apply the execution engine to real snow pit data from SnowPilot.

In [ ]:
# Parse a few sample pits
import warnings
import glob
warnings.filterwarnings('ignore')

# Get sample CAAML files from the data directory
caaml_files = glob.glob('data/snowpits-*.xml')[:10]  # Just 10 for demo

sample_pits = []
for file_path in caaml_files:
    try:
        snow_pit = parse_caaml_file(file_path)
        pit = Pit.from_snow_pit(snow_pit)
        sample_pits.append(pit)
    except Exception as e:
        continue

print(f"\nLoaded {len(sample_pits)} sample pits for demo")

In [ ]:
# Process each pit through the execution engine
pit_results = []

for i, pit in enumerate(sample_pits):
    try:
        # Create slabs from ECTP failures (new API)
        slabs = pit.create_slabs(weak_layer_def="ECTP_failure_layer")
        
        if not slabs:
            continue
        
        # Use first slab for demo
        slab = slabs[0]
        
        # Execute all pathways for elastic modulus
        results = engine.execute_all(
            slab=slab,
            target_parameter='elastic_modulus'
        )
        
        # Store summary
        pit_results.append({
            'pit_id': pit.pit_id or f'pit_{i}',
            'slab_id': slab.slab_id,
            'num_layers': len(slab.layers),
            'total_pathways': results.total_pathways,
            'successful_pathways': results.successful_pathways,
            'success_rate': results.successful_pathways / results.total_pathways if results.total_pathways > 0 else 0,
            'cache_hit_rate': results.cache_stats['hit_rate']
        })
        
    except Exception as e:
        pass

# Display results
if pit_results:
    pit_summary = pd.DataFrame(pit_results)
    print("SnowPilot Pit Processing Summary:\n")
    print(pit_summary.to_string(index=False))
    
    print(f"\n\nAggregate Statistics:")
    print(f"  Pits processed: {len(pit_results)}")
    print(f"  Average layers per pit: {pit_summary['num_layers'].mean():.1f}")
    print(f"  Average pathway success rate: {pit_summary['success_rate'].mean()*100:.1f}%")
    print(f"  Average cache hit rate: {pit_summary['cache_hit_rate'].mean()*100:.1f}%")
else:
    print("No pits were successfully processed.")

## 10. Visualizing Method Comparison

Let's create a visualization comparing elastic modulus estimates from different methods.

In [ ]:
# Use our test slab to compare methods
results = engine.execute_all(
    slab=test_slab,
    target_parameter='elastic_modulus'
)

# Collect E values by method for layers with computed values
plot_data = []
for pathway_desc, pathway_result in results.pathways.items():
    if pathway_result.success:
        density_method = pathway_result.methods_used.get('density', 'N/A')
        emod_method = pathway_result.methods_used.get('elastic_modulus', 'N/A')
        
        slab = pathway_result.slab
        for i, layer in enumerate(slab.layers):
            if layer.elastic_modulus is not None:
                plot_data.append({
                    'Layer': f"Layer {i}",
                    'Method': f"{density_method}\n+{emod_method}",
                    'E (MPa)': layer.elastic_modulus.nominal_value,
                    'E_err': layer.elastic_modulus.std_dev
                })

if plot_data:
    df_plot = pd.DataFrame(plot_data)
    
    # Create grouped bar chart
    fig, ax = plt.subplots(figsize=(14, 6))
    
    layers = df_plot['Layer'].unique()
    methods = df_plot['Method'].unique()
    
    x = np.arange(len(layers))
    width = 0.8 / len(methods)
    
    for i, method in enumerate(methods):
        method_data = df_plot[df_plot['Method'] == method]
        values = []
        errors = []
        for layer in layers:
            layer_method_data = method_data[method_data['Layer'] == layer]
            if not layer_method_data.empty:
                values.append(layer_method_data['E (MPa)'].values[0])
                errors.append(layer_method_data['E_err'].values[0])
            else:
                values.append(0)
                errors.append(0)
        
        ax.bar(x + i*width, values, width, label=method, yerr=errors, capsize=3)
    
    ax.set_xlabel('Layer')
    ax.set_ylabel('Elastic Modulus (MPa)')
    ax.set_title('Elastic Modulus Comparison Across Methods')
    ax.set_xticks(x + width * (len(methods) - 1) / 2)
    ax.set_xticklabels(layers)
    ax.legend(title='Density + E Method', bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("No data to plot.")

## Summary

This notebook demonstrated the key features of the Execution Engine:

1. **`ExecutionEngine(graph)`**: Initialize with the parameterization graph
2. **`list_available_pathways(target)`**: See all possible calculation pathways
3. **`execute_all(slab, target)`**: Run all pathways and get comprehensive results
4. **`execute_single(slab, target, methods)`**: Run a specific pathway

### Result Structure

- **`ExecutionResults`**: Top-level container with pathway_description → PathwayResult
- **`PathwayResult`**: Contains methods_used, layer_results, slab_result, success
- **`LayerResult`**: Contains layer with computed values and method_calls trace
- **`SlabResult`**: Contains plate theory parameters (A11, B11, D11, A55)

### Key Benefits

- **Comprehensive**: Automatically computes all possible method combinations
- **Traceable**: Full record of which methods produced which values
- **Flexible**: Handles missing data gracefully (skips silently)
- **Extensible**: Easy to add new methods to the graph